# Genji

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

base_url = 'https://ja.wikisource.org/wiki/%E6%BA%90%E6%B0%8F%E7%89%A9%E8%AA%9E_(%E6%B8%8B%E8%B0%B7%E6%A0%84%E4%B8%80%E6%A0%A1%E8%A8%82)'

In [2]:
base_soup = BeautifulSoup(requests.get(base_url).text)
ul = base_soup.find_all('ul')[1]

chapter_dict = {}
for li in ul.find_all('li'):
    chapter_dict[li.text] = f'https://ja.wikisource.org/{li.a["href"]}'


tog = pd.DataFrame.from_dict(chapter_dict, orient='index').reset_index().rename(columns={'index':'chapter',0:'link'})

In [3]:
def get_text(link):
    text = ''
    soup = BeautifulSoup(requests.get(link).text)
    for p in soup.find_all('p')[3:]:
        text += p.text.strip()
    return text

In [4]:
from tqdm import tqdm
tqdm.pandas()

tog['text'] = tog.link.progress_apply(get_text)

100%|██████████| 54/54 [00:15<00:00,  3.49it/s]


In [18]:
from konoha import SentenceTokenizer
tokenizer = SentenceTokenizer()

tog['sents'] = tog['text'].apply(tokenizer.tokenize)
tog_sents = tog[['chapter','sents']]
tog_sents = tog_sents.explode('sents').reset_index(drop=True)
tog_sents

,chapter,sents
0,桐壺,いづれの御時にか、女御、更衣あまたさぶらひたまひけるなかに、いとやむごとなき際にはあらぬが、...
1,桐壺,はじめより我はと思ひ上がりたまへる御方がた、めざましきものにおとしめ嫉みたまふ。
2,桐壺,同じほど、それより下臈の更衣たちは、ましてやすからず。
3,桐壺,朝夕の宮仕へにつけても、人の心をのみ動かし、恨みを負ふ積もりにやありけむ、いと篤しくなりゆき...
4,桐壺,上達部、上人なども、あいなく目を側めつつ、「いとまばゆき人の御おぼえなり。唐土にも、かかる事...
...,...,...
11055,夢浮橋,「いかが聞こえむ」など責められて、「心地のかき乱るやうにしはべるほど、ためらひて、今聞こえむ...
11056,夢浮橋,主人ぞ、この君に物語すこし聞こえて、「もののけにやおはすらむ。例のさまに見えたまふ折なく、悩...
11057,夢浮橋,所につけてをかしき饗応などしたれど、幼き心地は、そこはかとなくあわてたる心地して、「わざと奉...
11058,夢浮橋,人知れずゆかしき御ありさまをも、え見ずなりぬるを、おぼつかなく口惜しくて、心ゆかずながら参りぬ。


In [77]:
import spacy
nlp = spacy.load('ja_core_news_sm')

In [75]:
from spacy.tokens import DocBin

def spacyify(nlp, df,col_name,f_name='spacy_model_output'):
    texts = df[col_name].to_list()
    doc_bin = DocBin()
    for doc in tqdm(nlp.pipe(texts), total=len(texts)):
        doc_bin.add(doc)
    bytes_data = doc_bin.to_bytes()

    f = open(f'serialized_data/{f_name}','wb')
    f.write(bytes_data)
    f.close()

In [19]:
spacyify(nlp, tog_sents, 'sents')

100%|██████████| 11060/11060 [01:01<00:00, 181.11it/s]


In [20]:
tog_sents.to_csv('tog_sents.csv')

# Manyoshu

In [43]:
soup = BeautifulSoup(requests.get('https://jti.lib.virginia.edu/japanese/manyoshu/Man1Yos.html').text)
aoi = soup.center.td

In [59]:
from bs4 import NavigableString, Tag
for t in aoi:
    #if isinstance(tag.previous_element, NavigableString) and (tag.name == 'blockquote'):
    print('-----')
    print('prev:', t.previous_element, type(t.previous_element))
    print('curr:',t, type(t))
    print('next:',t.next_element, type(t.next_element))
    print('-----')

-----
prev: <td>
<center><h2><font color="#800000"><a name="第一巻">第一巻</a></font></h2></center>
<center><h1><font color="#800000"><b>雜歌</b></font></h1></center>
<br/><hr color="#800000" noshade="" size="2" width="80%"/><br/><br/>
<center><h2><font color="#800000"><a name="1">1</a></font></h2></center>
<p><font size="-1"><blockquote>[題詞]泊瀬朝倉宮御宇天皇代  [&lt;大&gt;泊瀬稚武天皇] / 天皇御製歌</blockquote></font></p><br/>
<p>[原文]篭毛與  美篭母乳  布久思毛與  美夫君志持  此岳尓  菜採須兒  家吉閑名  告&lt;紗&gt;根  虚見津  山跡乃國者  押奈戸手  吾許曽居  師&lt;吉&gt;名倍手  吾己曽座  我&lt;許&gt;背齒 告目  家呼毛名雄母</p>
<p>[訓読]篭もよ  み篭持ち  堀串もよ  み堀串持ち  この岡に  菜摘ます子  家聞かな  告らさね  そらみつ  大和の国は  おしなべて  我れこそ居れ  しきなべて  我れこそ座せ  我れこそば  告らめ  家をも名をも</p>
<p>[仮名],こもよ,みこもち,ふくしもよ,みぶくしもち,このをかに,なつますこ,いへきかな,のらさね,そらみつ,やまとのくには,おしなべて,われこそをれ,しきなべて,われこそませ,われこそば,のらめ,いへをもなをも</p>
<blockquote><font size="-1">
<p>[左注]</p></font>
<font size="-1">
<p>[校異]雑歌  [元][紀]  &lt;&gt; / 太 <img src="/japanese/manyoshu/rarr.gif"/> 大  [紀][冷][文] / 吉  [玉小琴](塙)(楓)  告 /沙 <img src="/japanese/manyoshu/rarr.gif"/> 紗 [元][類][冷]

In [152]:
import collections
poem_dict = collections.defaultdict(list)
for i,t in enumerate(aoi):
    if (isinstance(t.previous_element, NavigableString)) and (isinstance(t.next_element, Tag)) and (t.name == 'p'):
        for ele in t.next_siblings:
            if not isinstance(ele, NavigableString):
                if (ele.name == 'p') and (isinstance(ele.next_sibling, NavigableString)) and (ele.next_sibling.next_sibling.name == 'blockquote'):
                    poem_dict[f'{t.blockquote.text.strip()}_{i}'].append(ele.text)
                    break
                else:
                    poem_dict[f'{t.blockquote.text.strip()}_{i}'].append(ele.text)
        poem_dict[f'{t.blockquote.text.strip()}_{i}'] = [line for line in poem_dict[f'{t.blockquote.text.strip()}_{i}'] if len(line)>0]
poem_dict = dict(poem_dict)

In [158]:
pd.DataFrame.from_dict(poem_dict, orient='index').reset_index().rename(columns={'index':'title',0:'first',1:'second',2:'third'})

,title,first,second,third
0,[題詞]泊瀬朝倉宮御宇天皇代 [<大>泊瀬稚武天皇] / 天皇御製歌_12,[原文]篭毛與 美篭母乳 布久思毛與 美夫君志持 此岳尓 菜採須兒 家吉閑名 ...,[訓読]篭もよ み篭持ち 堀串もよ み堀串持ち この岡に 菜摘ます子 家聞かな ...,"[仮名],こもよ,みこもち,ふくしもよ,みぶくしもち,このをかに,なつますこ,いへきかな,の..."
1,[題詞]高市岡本宮御宇天皇代 [息長足日廣額天皇] / 天皇登香具山望國之時御製歌_28,[原文]山常庭 村山有等 取與呂布 天乃香具山 騰立 國見乎為者 國原波 煙立...,[訓読]大和には 群山あれど とりよろふ 天の香具山 登り立ち 国見をすれば 国...,"[仮名],やまとには,むらやまあれど,とりよろふ,あめのかぐやま,のぼりたち,くにみをすれば..."
2,[題詞]天皇遊猟内野之時中皇命使間人連老獻歌_44,[原文]八隅知之 我大王乃 朝庭 取撫賜 夕庭 伊縁立之 御執乃 梓弓之 奈...,[訓読]やすみしし 我が大君の 朝には 取り撫でたまひ 夕には い寄り立たしし ...,"[仮名],やすみしし,わがおほきみの,あしたには,とりなでたまひ,ゆふへには,いよりたたしし..."
3,[題詞]（天皇遊猟内野之時中皇命使間人連老獻歌）反歌_60,[原文]玉尅春 内乃大野尓 馬數而 朝布麻須等六 其草深野,[訓読]たまきはる宇智の大野に馬並めて朝踏ますらむその草深野,"[仮名],たまきはる,うちのおほのに,うまなめて,あさふますらむ,そのくさふかの"
4,[題詞]幸讃岐國安益郡之時軍王見山作歌_76,[原文]霞立 長春日乃 晩家流 和豆肝之良受 村肝乃 心乎痛見 奴要子鳥 卜歎...,[訓読]霞立つ 長き春日の 暮れにける わづきも知らず むらきもの 心を痛み ぬ...,"[仮名],かすみたつ,ながきはるひの,くれにける,わづきもしらず,むらきもの,こころをいたみ..."
...,...,...,...,...
79,[題詞]（或本従藤原<京>遷于寧樂宮時歌）反歌_1276,[原文]青丹吉 寧樂乃家尓者 万代尓 吾母将通 忘跡念勿,[訓読]あをによし奈良の家には万代に我れも通はむ忘ると思ふな,"[仮名],あをによし,ならのいへには,よろづよに,われもかよはむ,わするとおもふな"
80,[題詞]和銅五年壬子夏四月遣長田王于伊勢齊宮時山邊御井<作>歌_1292,[原文]山邊乃 御井乎見我弖利 神風乃 伊勢處女等 相見鶴鴨,[訓読]山辺の御井を見がてり神風の伊勢娘子どもあひ見つるかも,"[仮名],やまのへの,みゐをみがてり,かむかぜの,いせをとめども,あひみつるかも"
81,[題詞]（和銅五年壬子夏四月遣長田王于伊勢齊宮時山邊御井<作>歌）_1308,[原文]浦佐夫流 情佐麻<祢>之 久堅乃 天之四具礼能 流相見者,[訓読]うらさぶる心さまねしひさかたの天のしぐれの流らふ見れば,"[仮名],うらさぶる,こころさまねし,ひさかたの,あめのしぐれの,ながらふみれば"
82,[題詞]（和銅五年壬子夏四月遣長田王于伊勢齊宮時山邊御井<作>歌）_1324,[原文]海底 奥津白波 立田山 何時鹿越奈武 妹之當見武,[訓読]海の底沖つ白波龍田山いつか越えなむ妹があたり見む,"[仮名],わたのそこ,おきつしらなみ,たつたやま,いつかこえなむ,いもがあたりみむ"


In [203]:
def get_texts(url):
    soup = BeautifulSoup(requests.get(url).text)
    aoi = soup.center.td

    poem_dict = collections.defaultdict(list)
    for i,t in enumerate(aoi):
        if (isinstance(t.previous_element, NavigableString)) and (isinstance(t.next_element, Tag)) and (t.name == 'p'):
            for ele in t.next_siblings:
                if not isinstance(ele, NavigableString):
                    if (ele.name == 'p') and (isinstance(ele.next_sibling, NavigableString)) and (ele.next_sibling.next_sibling.name == 'blockquote'):
                        poem_dict[f'{t.blockquote.text.strip()}_{i}'].append(ele.text[4:])
                        break
                    else:
                        poem_dict[f'{t.blockquote.text.strip()}_{i}'].append(ele.text[4:])
            poem_dict[f'{t.blockquote.text.strip()}_{i}'] = [line for line in poem_dict[f'{t.blockquote.text.strip()}_{i}'] if len(line)>0]
    poem_dict = dict(poem_dict)

    return pd.DataFrame.from_dict(poem_dict, orient='index').reset_index().rename(columns={'index':'title',0:'manyogana',1:'kundoku',2:'kana'})

In [206]:
DFs = []
for i in range(1, 21):
    if i < 10:
        DFs.append(get_texts(f'https://jti.lib.virginia.edu/japanese/manyoshu/Man{i}Yos.html'))
    else:
        DFs.append(get_texts(f'https://jti.lib.virginia.edu/japanese/manyoshu/Man{i}Yo.html'))
manyoshu = pd.concat(DFs).dropna()

In [213]:
ex = manyoshu.loc[manyoshu.manyogana.str.contains('空蝉'), 'manyogana'].iloc[1]

In [214]:
spacy_doc = nlp(ex)
search = '空蝉'
for token in spacy_doc:
    if ((token.lemma_.lower() == search) or (token.lemma_.title() == search)) and (token.head.pos_ != 'PUNCT'):
        print(
            (token.text, token.pos_), 
            (token.head.text, token.head.pos_)
            )
        print(spacy_doc.text)

('空蝉', 'NOUN') ('勝者', 'NOUN')
空蝉師  神尓不勝者  離居而  朝嘆君  放居而  吾戀君  玉有者  手尓巻持而  衣有者  脱時毛無  吾戀  君曽伎賊乃夜  夢所見鶴


In [215]:
spacyify(nlp, manyoshu, 'manyogana', f_name='manyoshu_spacy_output')

100%|██████████| 4560/4560 [00:13<00:00, 331.13it/s]


In [216]:
manyoshu.to_csv('manyoshu.csv')

# Kokinshu

In [6]:
from bs4 import BeautifulSoup
from bs4 import NavigableString, Tag
import requests
import re
tei = BeautifulSoup(requests.get('https://jti.lib.virginia.edu/japanese/kokinshu/kikokin.html').text).teiheader

In [7]:
import collections

poem_dict = collections.defaultdict(str)
for center in tei.find_all('center')[1:]:
    if center.text.strip().isdigit():
        num = center.text.strip()
        sib_list = list(center.next_siblings)
        text = ''
        if sib_list[3].name == 'center': # if title
            if sib_list[7].name == 'blockquote': # if blockquote
                title = sib_list[3].text.strip()
                first = sib_list[7].text.strip()
                second = str(sib_list[10]).strip()
            else:
                title = sib_list[3].text.strip()
                first = ''
                second = str(sib_list[6]).strip()
        else:
            if sib_list[3].name == 'blockquote':
                title = ''
                first = sib_list[3].text.strip()
                try:
                    second = str(sib_list[6]).strip()    
                except:
                    continue
            else:
                title = ''
                first = ''
                second = str(sib_list[2]).strip()
        poem_dict[f'{title}_{num}'] = f'{first}\n{second}'.strip()

In [8]:
import pandas as pd
poem_dict = dict(poem_dict)
kokinshu = pd.DataFrame.from_dict(poem_dict, orient='index').reset_index().rename(columns={'index':'title',0:'text'})

In [9]:
kokinshu

,title,text
0,在原元方_1,ふるとしに春たちける日よめる\nとしのうちに春はきにけりひととせをこぞとやいはむことしとやいはむ
1,紀貫之_2,はるたちける日よめる\n袖ひちてむすびし水のこほれるを春立つけふの風やとくらむ
2,よみ人しらず_3,題しらず\n春霞たてるやいづこみよしののよしのの山に雪はふりつつ
3,_4,二条のきさきのはるのはじめの御うた\n雪の内に春はきにけりうぐひすのこほれる涙今やとくらむ
4,よみ人しらず_5,題しらず\n梅がえにきゐるうぐひすはるかけてなけどもいまだ雪はふりつつ
...,...,...
1105,_1107,わぎもこにあふさか山のしのすすきほにはいでずもこひわたるかな
1106,_1108,こひしくはしたにを思へ紫の、下\nいぬがみのとこの山なるなとり河いさとこたへよわがなもらすな
1107,うねめのたてまつれる_1109,返し\n山しなのおとはのたきのおとにのみ人のしるべくわがこひめやも
1108,_1110,思ふてふことのはのみや秋をへて、下 \nそとほりひめのひとりゐてみかどをこひたてまつりて


In [10]:
kokinshu.to_csv('kokinshu.csv')

In [81]:
from konoha import SentenceTokenizer
tokenizer = SentenceTokenizer()

kokinshu['sents'] = kokinshu['text'].apply(tokenizer.tokenize)
kokinshu_sents = kokinshu[['title','sents']]
kokinshu_sents = kokinshu_sents.explode('sents').reset_index(drop=True)
kokinshu_sents = kokinshu_sents.dropna() 

In [82]:
from tqdm import tqdm
tqdm.pandas()
spacyify(nlp, kokinshu_sents, 'sents', f_name='kokinshu_spacy_output')

100%|██████████| 1699/1699 [00:03<00:00, 560.17it/s]


In [83]:
kokinshu_sents.to_csv('kokinshu_sents.csv')

In [84]:
kokinshu

,title,text,sents
0,在原元方_1,ふるとしに春たちける日よめる\nとしのうちに春はきにけりひととせをこぞとやいはむことしとやいはむ,"[ふるとしに春たちける日よめる, としのうちに春はきにけりひととせをこぞとやいはむことしとや..."
1,紀貫之_2,はるたちける日よめる\n袖ひちてむすびし水のこほれるを春立つけふの風やとくらむ,"[はるたちける日よめる, 袖ひちてむすびし水のこほれるを春立つけふの風やとくらむ]"
2,よみ人しらず_3,題しらず\n春霞たてるやいづこみよしののよしのの山に雪はふりつつ,"[題しらず, 春霞たてるやいづこみよしののよしのの山に雪はふりつつ]"
3,_4,二条のきさきのはるのはじめの御うた\n雪の内に春はきにけりうぐひすのこほれる涙今やとくらむ,"[二条のきさきのはるのはじめの御うた, 雪の内に春はきにけりうぐひすのこほれる涙今やとくらむ]"
4,よみ人しらず_5,題しらず\n梅がえにきゐるうぐひすはるかけてなけどもいまだ雪はふりつつ,"[題しらず, 梅がえにきゐるうぐひすはるかけてなけどもいまだ雪はふりつつ]"
...,...,...,...
1105,_1107,わぎもこにあふさか山のしのすすきほにはいでずもこひわたるかな,[わぎもこにあふさか山のしのすすきほにはいでずもこひわたるかな]
1106,_1108,こひしくはしたにを思へ紫の、下\nいぬがみのとこの山なるなとり河いさとこたへよわがなもらすな,"[こひしくはしたにを思へ紫の、下, いぬがみのとこの山なるなとり河いさとこたへよわがなもらすな]"
1107,うねめのたてまつれる_1109,返し\n山しなのおとはのたきのおとにのみ人のしるべくわがこひめやも,"[返し, 山しなのおとはのたきのおとにのみ人のしるべくわがこひめやも]"
1108,_1110,思ふてふことのはのみや秋をへて、下 \nそとほりひめのひとりゐてみかどをこひたてまつりて,"[思ふてふことのはのみや秋をへて、下, そとほりひめのひとりゐてみかどをこひたてまつりて]"


In [85]:
pd.read_csv('manyoshu.csv')

,Unnamed: 0,title,manyogana,kundoku,kana
0,0,[題詞]泊瀬朝倉宮御宇天皇代 [<大>泊瀬稚武天皇] / 天皇御製歌_12,篭毛與 美篭母乳 布久思毛與 美夫君志持 此岳尓 菜採須兒 家吉閑名 告<紗>...,篭もよ み篭持ち 堀串もよ み堀串持ち この岡に 菜摘ます子 家聞かな 告らさ...,",こもよ,みこもち,ふくしもよ,みぶくしもち,このをかに,なつますこ,いへきかな,のらさね,..."
1,1,[題詞]高市岡本宮御宇天皇代 [息長足日廣額天皇] / 天皇登香具山望國之時御製歌_29,山常庭 村山有等 取與呂布 天乃香具山 騰立 國見乎為者 國原波 煙立龍 海...,大和には 群山あれど とりよろふ 天の香具山 登り立ち 国見をすれば 国原は ...,",やまとには,むらやまあれど,とりよろふ,あめのかぐやま,のぼりたち,くにみをすれば,くには..."
2,2,[題詞]天皇遊猟内野之時中皇命使間人連老獻歌_46,八隅知之 我大王乃 朝庭 取撫賜 夕庭 伊縁立之 御執乃 梓弓之 奈加弭乃 ...,やすみしし 我が大君の 朝には 取り撫でたまひ 夕には い寄り立たしし み執らし...,",やすみしし,わがおほきみの,あしたには,とりなでたまひ,ゆふへには,いよりたたしし,みとら..."
3,3,[題詞]（天皇遊猟内野之時中皇命使間人連老獻歌）反歌_63,玉尅春 内乃大野尓 馬數而 朝布麻須等六 其草深野,たまきはる宇智の大野に馬並めて朝踏ますらむその草深野,",たまきはる,うちのおほのに,うまなめて,あさふますらむ,そのくさふかの"
4,4,[題詞]幸讃岐國安益郡之時軍王見山作歌_80,霞立 長春日乃 晩家流 和豆肝之良受 村肝乃 心乎痛見 奴要子鳥 卜歎居者 ...,霞立つ 長き春日の 暮れにける わづきも知らず むらきもの 心を痛み ぬえこ鳥 ...,",かすみたつ,ながきはるひの,くれにける,わづきもしらず,むらきもの,こころをいたみ,ぬえこ..."
...,...,...,...,...,...
4555,219,[題詞]（属目山齊作歌三首）_3735,伊氣美豆尓 可氣左倍見要Ｃ 佐伎尓保布 安之婢乃波奈乎 蘇弖尓古伎礼奈,池水に影さへ見えて咲きにほふ馬酔木の花を袖に扱入れな,",いけみづに,かげさへみえて,さきにほふ,あしびのはなを,そでにこきれな"
4556,220,[題詞]（属目山齊作歌三首）_3752,伊蘇可氣乃 美由流伊氣美豆 Ｃ流麻Ｏ尓 左家流安之婢乃 知良麻久乎思母,礒影の見ゆる池水照るまでに咲ける馬酔木の散らまく惜しも,",いそかげの,みゆるいけみづ,てるまでに,さけるあしびの,ちらまくをしも"
4557,221,[題詞]二月十日於内相宅餞渤海大使小野田守朝臣等宴歌一首_3769,阿乎宇奈波良 加是奈美奈妣伎 由久左久佐 都々牟許等奈久 布祢波々夜家無,青海原風波靡き行くさ来さつつむことなく船は速けむ,",あをうなはら,かぜなみなびき,ゆくさくさ,つつむことなく,ふねははやけむ"
4558,222,[題詞]七月五日於<治>部少輔大原今城真人宅餞因幡守大伴宿祢家持宴歌一首_3786,秋風乃 須恵布伎奈婢久 波疑能花 登毛尓加射左受 安比加和可礼牟,秋風の末吹き靡く萩の花ともにかざさず相か別れむ,",あきかぜの,すゑふきなびく,はぎのはな,ともにかざさず,あひかわかれむ"


# Data gen

In [15]:
import pandas as pd
from spacy.tokens import DocBin
import spacy

nlp = spacy.load('ja_core_news_sm')
m = pd.read_csv('manyoshu.csv')
m_bytes_file = open('serialized_data/manyoshu_spacy_output','rb').read()
doc_bin = DocBin().from_bytes(m_bytes_file)
m['spacy_docs'] =  pd.Series(doc_bin.get_docs(nlp.vocab))
m

,Unnamed: 0,title,manyogana,kundoku,kana,spacy_docs
0,0,[題詞]泊瀬朝倉宮御宇天皇代 [<大>泊瀬稚武天皇] / 天皇御製歌_12,篭毛與 美篭母乳 布久思毛與 美夫君志持 此岳尓 菜採須兒 家吉閑名 告<紗>...,篭もよ み篭持ち 堀串もよ み堀串持ち この岡に 菜摘ます子 家聞かな 告らさ...,",こもよ,みこもち,ふくしもよ,みぶくしもち,このをかに,なつますこ,いへきかな,のらさね,...","(篭, 毛, 與, , 美, 篭, 母乳, , 布, 久, 思, 毛, 與, , 美夫..."
1,1,[題詞]高市岡本宮御宇天皇代 [息長足日廣額天皇] / 天皇登香具山望國之時御製歌_29,山常庭 村山有等 取與呂布 天乃香具山 騰立 國見乎為者 國原波 煙立龍 海...,大和には 群山あれど とりよろふ 天の香具山 登り立ち 国見をすれば 国原は ...,",やまとには,むらやまあれど,とりよろふ,あめのかぐやま,のぼりたち,くにみをすれば,くには...","(山, 常, 庭, , 村山, 有等, , 取, 與, 呂布, , 天乃, 香具, 山..."
2,2,[題詞]天皇遊猟内野之時中皇命使間人連老獻歌_46,八隅知之 我大王乃 朝庭 取撫賜 夕庭 伊縁立之 御執乃 梓弓之 奈加弭乃 ...,やすみしし 我が大君の 朝には 取り撫でたまひ 夕には い寄り立たしし み執らし...,",やすみしし,わがおほきみの,あしたには,とりなでたまひ,ゆふへには,いよりたたしし,みとら...","(八隅, 知之, , 我, 大王, 乃, , 朝庭, , 取, 撫賜, , 夕, 庭..."
3,3,[題詞]（天皇遊猟内野之時中皇命使間人連老獻歌）反歌_63,玉尅春 内乃大野尓 馬數而 朝布麻須等六 其草深野,たまきはる宇智の大野に馬並めて朝踏ますらむその草深野,",たまきはる,うちのおほのに,うまなめて,あさふますらむ,そのくさふかの","(玉, 尅春, , 内, 乃, 大野, 尓, , 馬, 數, 而, , 朝布, 麻, ..."
4,4,[題詞]幸讃岐國安益郡之時軍王見山作歌_80,霞立 長春日乃 晩家流 和豆肝之良受 村肝乃 心乎痛見 奴要子鳥 卜歎居者 ...,霞立つ 長き春日の 暮れにける わづきも知らず むらきもの 心を痛み ぬえこ鳥 ...,",かすみたつ,ながきはるひの,くれにける,わづきもしらず,むらきもの,こころをいたみ,ぬえこ...","(霞, 立, , 長春, 日, 乃, , 晩, 家, 流, , 和, 豆, 肝, 之,..."
...,...,...,...,...,...,...
4555,219,[題詞]（属目山齊作歌三首）_3735,伊氣美豆尓 可氣左倍見要Ｃ 佐伎尓保布 安之婢乃波奈乎 蘇弖尓古伎礼奈,池水に影さへ見えて咲きにほふ馬酔木の花を袖に扱入れな,",いけみづに,かげさへみえて,さきにほふ,あしびのはなを,そでにこきれな","(伊, 氣, 美豆, 尓, , 可, 氣, 左, 倍, 見, 要, Ｃ, , 佐伎, 尓..."
4556,220,[題詞]（属目山齊作歌三首）_3752,伊蘇可氣乃 美由流伊氣美豆 Ｃ流麻Ｏ尓 左家流安之婢乃 知良麻久乎思母,礒影の見ゆる池水照るまでに咲ける馬酔木の散らまく惜しも,",いそかげの,みゆるいけみづ,てるまでに,さけるあしびの,ちらまくをしも","(伊蘇, 可, 氣, 乃, , 美由, 流, 伊, 氣, 美豆, , Ｃ, 流, 麻, ..."
4557,221,[題詞]二月十日於内相宅餞渤海大使小野田守朝臣等宴歌一首_3769,阿乎宇奈波良 加是奈美奈妣伎 由久左久佐 都々牟許等奈久 布祢波々夜家無,青海原風波靡き行くさ来さつつむことなく船は速けむ,",あをうなはら,かぜなみなびき,ゆくさくさ,つつむことなく,ふねははやけむ","(阿乎, 宇奈, 波良, , 加, 是, 奈, 美奈, 妣, 伎, , 由久, 左, 久..."
4558,222,[題詞]七月五日於<治>部少輔大原今城真人宅餞因幡守大伴宿祢家持宴歌一首_3786,秋風乃 須恵布伎奈婢久 波疑能花 登毛尓加射左受 安比加和可礼牟,秋風の末吹き靡く萩の花ともにかざさず相か別れむ,",あきかぜの,すゑふきなびく,はぎのはな,ともにかざさず,あひかわかれむ","(秋, 風乃, , 須恵, 布, 伎, 奈, 婢, 久, , 波, 疑, 能, 花, ..."


In [20]:
from tqdm import tqdm
def spacyify(nlp, df,col_name,f_name='spacy_model_output'):
    texts = df[col_name].to_list()
    doc_bin = DocBin()
    for doc in tqdm(nlp.pipe(texts), total=len(texts)):
        doc_bin.add(doc)
    bytes_data = doc_bin.to_bytes()

    f = open(f'serialized_data/{f_name}','wb')
    f.write(bytes_data)
    f.close()

In [23]:
k = pd.read_csv('kokinshu.csv').dropna()
k

,Unnamed: 0,title,text
0,0,在原元方_1,ふるとしに春たちける日よめる\nとしのうちに春はきにけりひととせをこぞとやいはむことしとやいはむ
1,1,紀貫之_2,はるたちける日よめる\n袖ひちてむすびし水のこほれるを春立つけふの風やとくらむ
2,2,よみ人しらず_3,題しらず\n春霞たてるやいづこみよしののよしのの山に雪はふりつつ
3,3,_4,二条のきさきのはるのはじめの御うた\n雪の内に春はきにけりうぐひすのこほれる涙今やとくらむ
4,4,よみ人しらず_5,題しらず\n梅がえにきゐるうぐひすはるかけてなけどもいまだ雪はふりつつ
...,...,...,...
1105,1105,_1107,わぎもこにあふさか山のしのすすきほにはいでずもこひわたるかな
1106,1106,_1108,こひしくはしたにを思へ紫の、下\nいぬがみのとこの山なるなとり河いさとこたへよわがなもらすな
1107,1107,うねめのたてまつれる_1109,返し\n山しなのおとはのたきのおとにのみ人のしるべくわがこひめやも
1108,1108,_1110,思ふてふことのはのみや秋をへて、下 \nそとほりひめのひとりゐてみかどをこひたてまつりて


In [24]:
spacyify(nlp, k, 'text', f_name='kokinshu_model_output')

100%|██████████| 1109/1109 [00:02<00:00, 376.39it/s]


In [36]:
k_bytes_file = open('serialized_data/kokinshu_spacy_output','rb').read()
doc_bin = DocBin().from_bytes(k_bytes_file)
k['spacy_docs'] =  pd.Series(doc_bin.get_docs(nlp.vocab))
k

,Unnamed: 0,title,text,spacy_docs
0,0,在原元方_1,ふるとしに春たちける日よめる\nとしのうちに春はきにけりひととせをこぞとやいはむことしとやいはむ,"(ふる, と, し, に, 春, たち, ける, 日, よめる)"
1,1,紀貫之_2,はるたちける日よめる\n袖ひちてむすびし水のこほれるを春立つけふの風やとくらむ,"(とし, の, うち, に, 春, は, き, に, けり, ひと, とせ, を, こぞ, ..."
2,2,よみ人しらず_3,題しらず\n春霞たてるやいづこみよしののよしのの山に雪はふりつつ,"(はる, たち, ける, 日, よめる)"
3,3,_4,二条のきさきのはるのはじめの御うた\n雪の内に春はきにけりうぐひすのこほれる涙今やとくらむ,"(袖, ひち, て, むすび, し, 水, の, こほ, れる, を, 春, 立つ, けふ,..."
4,4,よみ人しらず_5,題しらず\n梅がえにきゐるうぐひすはるかけてなけどもいまだ雪はふりつつ,"(題, しら, ず)"
...,...,...,...,...
1105,1105,_1107,わぎもこにあふさか山のしのすすきほにはいでずもこひわたるかな,"(たえ, ず, ゆく, あすか, の, 河, の, よどみ, な, ば, 心, ある, と,..."
1106,1106,_1108,こひしくはしたにを思へ紫の、下\nいぬがみのとこの山なるなとり河いさとこたへよわがなもらすな,"(よど, 河, の, よどむ, と, 人, は, 見る, らめ, ど, 流れ, て, ふか,..."
1107,1107,うねめのたてまつれる_1109,返し\n山しなのおとはのたきのおとにのみ人のしるべくわがこひめやも,"(そこひ, なき, ふち, や, は, さわぐ, 山河, の, あさ, きせ, に, こそ,..."
1108,1108,_1110,思ふてふことのはのみや秋をへて、下 \nそとほりひめのひとりゐてみかどをこひたてまつりて,"(紅, の, はつ, 花, ぞ, め, の, 色, ふかく, 思ひ, し, 心, 我, わす..."


In [44]:
def find_utsusemi(raw_title, spacy_doc):
    if len(raw_title.split('_')[0]) > 0:
        title = f'{raw_title.split("_")[0].strip()} -- Poem {raw_title.split("_")[1]}'
    else:
        title = f'Poem {raw_title.split("_")[1]}'
    
    for token in spacy_doc:
        if ((token.text.lower() == '空蝉') or (token.text.title() == '空蝉')) and (token.head.pos_ != 'PUNCT'):
            print(title)
            print(spacy_doc.text)
            print('------------')

In [46]:
p = k.apply(lambda x: find_utsusemi(x['title'], x['spacy_docs']),axis=1)

そせい -- Poem 126
空蝉の世にもにたるか花ざくらさくと見しまにかつちりにけり
------------
さだののぼる -- Poem 769
空蝉のからは木ごとにとどむれどたまのゆくへを見ぬぞかなしき
------------
つらゆき -- Poem 1103
空蝉の世の人ごとのしげければわすれぬもののかれぬべらなり
------------


In [47]:
p = m.apply(lambda x: find_utsusemi(x['title'], x['spacy_docs']),axis=1)

[題詞]（麻續王流於伊勢國伊良虞嶋之時人哀傷作歌）麻續王聞之感傷和歌 -- Poem 403
空蝉之  命乎惜美  浪尓所濕  伊良虞能嶋之  玉藻苅食
------------
[題詞]天皇崩時婦人作歌一首  [姓氏未詳] -- Poem 1124
空蝉師  神尓不勝者  離居而  朝嘆君  放居而  吾戀君  玉有者  手尓巻持而  衣有者  脱時毛無  吾戀  君曽伎賊乃夜  夢所見鶴
------------
[題詞]大伴坂上郎女怨恨歌一首[并短歌] -- Poem 2307
押照  難波乃菅之  根毛許呂尓  君之聞四<手>  年深  長四云者  真十鏡  磨師情乎  縦手師  其日之極  浪之共  靡珠藻乃  云々  意者不持  大船乃  憑有時丹  千磐破  神哉将離  空蝉乃  人歟禁良武  通為  君毛不来座  玉梓之  使母不所見  成奴礼婆  痛毛為便無三  夜干玉乃  夜者須我良尓  赤羅引  日母至闇  雖嘆  知師乎無三  雖念  田付乎白二  幼婦常  言雲知久  手小童之  哭耳泣管  俳徊  君之使乎  待八兼手六
------------
[題詞]（又大伴宿祢家持和歌三首） -- Poem 4245
空蝉乃  代也毛二行  何為跡鹿  妹尓不相而  吾獨将宿
------------
[題詞]（詠花） -- Poem 777
毎年  梅者開友  空蝉之  <世>人君羊蹄  春無有来
------------
[題詞]（問答歌） -- Poem 4715
空蝉之  人目乎繁  不相而  年之經者  生跡毛奈思
------------
[題詞]（問答歌） -- Poem 4732
空蝉之  人目繁者  夜干玉之  夜夢乎  次而所見欲
------------
[題詞] -- Poem 1995
高山  与海社者  山随  如此毛現  海随  然真有目  人者<花>物曽  空蝉与人 
------------
